# Reporting
## Create daily full list

In [ ]:
import pandas as pd
from shutil import copytree
from shutil import rmtree
import hubmapbags
import pathlib
import json
import yaml

from pathlib import Path
import pandas as pd
from tqdm import tqdm
from warnings import warn as warning
from datetime import datetime
from time import sleep
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

instance = 'prod'
token = ''

In [ ]:
assay_names = hubmapbags.apis.get_assay_types( token=token )
print(assay_names)

In [ ]:
now = datetime.now()
output_directory = '.data'
if not Path(output_directory).exists():
    Path(output_directory).mkdir()

report_output_directory = 'daily-report'
if not Path(report_output_directory).exists():
    Path(report_output_directory).mkdir()

report_output_filename = report_output_directory + '/' + str(now.strftime('%Y%m%d')) + '.tsv'
print(report_output_filename)

In [ ]:
def is_primary( hubmap_id, instance='prod', token=None ):
    metadata = hubmapbags.apis.get_ancestors_info( hubmap_id, instance=instance, token=token )
    if 'entity_type' in metadata[0].keys() and  metadata[0]['entity_type'] == 'Sample':
        return True
    else:
        if 'error' in metadata[0]:
            warning(metadata[0]['error'])
        return False
    
def has_metadata( metadata ):
    if 'ingest_metadata' in metadata.keys() and 'metadata' in metadata['ingest_metadata'].keys():
        return True
    else:
        return False

In [ ]:
if not Path(report_output_filename).exists():
    # get assay types
    assay_names = hubmapbags.apis.get_assay_types( token=token )
    
    report = pd.DataFrame()
    for assay_name in assay_names:
        datasets = pd.DataFrame(hubmapbags.get_ids( assay_name=assay_name, token=token ))
        datasets['assay'] = assay_name

        if datasets.empty:
            continue
            
        report = pd.concat([report,datasets], axis=0)
        
        #for index, datum in tqdm(datasets.iterrows()):
        #    datasets.loc[index, 'directory'] = hubmapbags.apis.get_directory( datum['hubmap_id'], instance='prod', token=token )
        #    metadata = hubmapbags.apis.get_dataset_info( datum['hubmap_id'], instance='prod', token=token )
            
    #report = report[['group_name','uuid','hubmap_id','status','is_protected','data_type','assay_type','assay_category','directory']]
    #report.to_csv( report_output_filename, sep='\t', index=False )
    #report.to_pickle( report_output_filename.replace('tsv','pkl') )
else:
    print('File found on disk. Loading ' + report_output_filename + '.' )
    report = pd.read_pickle( report_output_filename.replace('tsv', 'pkl') )

In [ ]:
            if has_metadata( metadata ):
                datasets.loc['has_metadata'] = True
                if 'ingest_metadata' in metadata.keys():
                    datasets.loc[index,'assay_type'] = metadata['ingest_metadata']['metadata']['assay_type']
                    datasets.loc[index,'assay_category'] = metadata['ingest_metadata']['metadata']['assay_category']
                else:
                    datasets.loc[index,'assay_type'] = None
                    datasets.loc[index, 'assay_category'] = None
            else:
                datasets.loc['has_metadata'] = False
            
        report = pd.concat([report,datasets])
    

In [ ]:
def __get_directory( hubmap_id ):
    try:
        return hubmapbags.apis.get_directory( hubmap_id, instance='prod', token=token )
    except:
        return None
    
%time report['directory'] = report['hubmap_id'].parallel_apply(__get_directory)

In [ ]:


def __is_primary( hubmap_id ):
    try:
        return hubmapbags.apis.is_primary( hubmap_id, instance='prod', token=token )
    except:
        return None
    
%time report['directory'] = report['hubmap_id'].paralel_apply(__get_directory)